In [1]:
import math
from tensorboardX import SummaryWriter
sess = None

In [2]:
import tensorflow as tf
import collections
gpu_options = tf.GPUOptions(allow_growth=True,per_process_gpu_memory_fraction=0.8)
tf.reset_default_graph()
sess = tf.InteractiveSession(config=tf.ConfigProto(gpu_options=gpu_options))

In [3]:
import gym
env = gym.make('CartPole-v1')
observation_shape = env.observation_space.shape
n_actions = env.action_space.n
shape =(None, ) + observation_shape
print(shape)

(None, 4)


/home/trrrrr/anaconda3/envs/rl/lib/python3.6/site-packages/gym/envs/registration.py:14: PkgResourcesDeprecationWarning: Parameters to load are deprecated.  Call .resolve and .require separately.
  result = entry_point.load(False)


In [4]:
# The typical imports
import gym
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

# Imports specifically so we can render outputs in Jupyter.
from JSAnimation.IPython_display import display_animation
from matplotlib import animation
from IPython.display import display


def display_frames_as_gif(frames):
    """
    Displays a list of frames as a gif, with controls
    """
    #plt.figure(figsize=(frames[0].shape[1] / 72.0, frames[0].shape[0] / 72.0), dpi = 72)
    patch = plt.imshow(frames[0])
    plt.axis('off')

    def animate(i):
        patch.set_data(frames[i])

    anim = animation.FuncAnimation(plt.gcf(), animate, frames = len(frames), interval=50)
    display(display_animation(anim, default_mode='loop'))

In [5]:
class EnvCreator(object):
    def __init__(self, name):
        self.name = name
    def __call__(self, ):
        return gym.make(self.name)

In [6]:
from a2c import A2CAgent
import tr_helpers
import networks
import ray
ray.init()
a2c_cartpole_config = {
    'GAMMA' : 0.99,
    'LEARNING_RATE' : 1e-3,
    'NAME' : 'A2C',
    'SCORE_TO_WIN' : 200,
    'NETWORK' : networks.CartPoleA2C(),
    'REWARD_SHAPER' : tr_helpers.DefaultRewardsShaper(),
    'EPISODES_TO_LOG' : 20, 
    'LIVES_REWARD' : 5,
    'STEPS_NUM' : 3,
    'ENTROPY_COEF' : 0.001,
    'NUM_ACTORS' : 8,
    'ENV' : EnvCreator('CartPole-v1')
}
#self, sess, env_name, observation_shape, actions_num, config = default_config
observation_shape = env.observation_space.shape
actions_num = env.action_space.n
print(observation_shape)
agent = A2CAgent(sess,'cartpole', observation_shape, actions_num, a2c_cartpole_config)
agent.train()

2019-04-08 20:53:53,558	INFO node.py:278 -- Process STDOUT and STDERR is being redirected to /tmp/ray/session_2019-04-08_20-53-53_25037/logs.
2019-04-08 20:53:53,675	INFO services.py:396 -- Waiting for redis server at 127.0.0.1:29793 to respond...
2019-04-08 20:53:53,831	INFO services.py:396 -- Waiting for redis server at 127.0.0.1:28630 to respond...
2019-04-08 20:53:53,841	INFO services.py:798 -- Starting Redis shard with 10.0 GB max memory.
2019-04-08 20:53:53,906	INFO services.py:1360 -- Starting the Plasma object store with 13.477717606 GB memory using /dev/shm.



View the web UI at http://localhost:8890/notebooks/ray_ui.ipynb?token=cb79c37d4100f235821ccb562a79686fa5f3d98aa544fd0b

(4,)
9.0
10.0
14.0
16.0
18.0
18.0
11.0
23.0
13.0
13.0
10.0
9.0
19.0
20.0
16.0
23.0
10.0
11.0
51.0
28.0
12.0
14.0
11.0
21.0
8.0
12.0
15.0
18.0
12.0
18.0
34.0
10.0
18.0
19.0
18.0
10.0
10.0
8.0
8.0
9.0
25.0
14.0
15.0
14.0
a_loss 0.7179389
c_loss 2.8671145
entropy 0.44275993
37.0
7.0
8.0
13.0
8.0
10.0
16.0
19.0
10.0
9.0
8.0
8.0
9.0
9.0
16.0
9.0
7.0
9.0
10.0
10.0
9.0
14.0
8.0
11.0
8.0
8.0
12.0
13.0
11.0
9.0
7.0
11.0
9.0
10.0
9.0
9.0
9.0
11.0
13.0
11.0
9.0
8.0
9.0
8.0
11.0
9.0
8.0
10.0
9.0
9.0
9.0
7.0
7.0
7.0
10.0
10.0
10.0
9.0
10.0
9.0
9.0
9.0
9.0
8.0
7.0
12.0
8.0
10.0
11.0
9.0
8.0
9.0
13.0
8.0
8.0
a_loss 0.0445887
c_loss 53.75568
entropy 0.06811067
9.0
9.0
8.0
8.0
9.0
8.0
9.0
10.0
7.0
7.0
9.0
7.0
8.0
8.0
9.0
10.0
8.0
8.0
8.0
12.0
10.0
8.0
7.0
8.0
8.0
9.0
8.0
9.0
9.0
8.0
15.0
7.0
9.0
9.0
8.0
8.0
13.0
15.0
8.0
10.0
10.0
7.0
7.0
7.0
9.0
10.0
8.0
9.0
8.0
10.0
8.0
8.0
7.0
7.0

7.0
10.0
9.0
7.0
9.0
10.0
8.0
7.0
8.0
11.0
9.0
8.0
8.0
9.0
8.0
9.0
8.0
9.0
9.0
8.0
7.0
10.0
9.0
11.0
9.0
9.0
8.0
9.0
8.0
8.0
10.0
8.0
8.0
9.0
8.0
7.0
8.0
9.0
8.0
8.0
8.0
8.0
9.0
8.0
a_loss 0.39321056
c_loss 0.53622675
entropy 0.0813393
10.0
9.0
9.0
8.0
8.0
9.0
8.0
9.0
9.0
8.0
10.0
8.0
9.0
8.0
9.0
9.0
7.0
7.0
9.0
7.0
11.0
9.0
9.0
11.0
8.0
8.0
8.0
9.0
8.0
10.0
8.0
9.0
9.0
7.0
7.0
9.0
9.0
8.0
7.0
8.0
8.0
9.0
9.0
9.0
9.0
9.0
8.0
10.0
8.0
10.0
9.0
9.0
9.0
8.0
7.0
9.0
7.0
7.0
8.0
9.0
7.0
9.0
8.0
8.0
8.0
8.0
7.0
9.0
8.0
8.0
8.0
8.0
9.0
9.0
9.0
8.0
9.0
9.0
10.0
9.0
10.0
8.0
9.0
9.0
a_loss -0.023389079
c_loss 9.374237
entropy 0.10357009
9.0
8.0
8.0
8.0
9.0
9.0
8.0
9.0
7.0
9.0
10.0
7.0
8.0
10.0
7.0
8.0
9.0
9.0
8.0
8.0
8.0
11.0
9.0
7.0
8.0
8.0
8.0
9.0
10.0
9.0
8.0
9.0
9.0
8.0
11.0
7.0
9.0
9.0
9.0
8.0
8.0
9.0
8.0
8.0
10.0
8.0
11.0
9.0
9.0
9.0
9.0
9.0
9.0
8.0
8.0
9.0
8.0
8.0
11.0
8.0
8.0
9.0
8.0
9.0
9.0
8.0
8.0
9.0
8.0
8.0
9.0
12.0
9.0
8.0
7.0
8.0
9.0
8.0
10.0
9.0
7.0
7.0
8.0
7.0
a_loss 0.011367162

8.0
8.0
9.0
10.0
10.0
9.0
9.0
9.0
8.0
8.0
8.0
9.0
7.0
8.0
7.0
9.0
9.0
9.0
9.0
9.0
10.0
8.0
9.0
9.0
9.0
8.0
8.0
9.0
9.0
9.0
7.0
9.0
8.0
7.0
8.0
8.0
9.0
8.0
9.0
9.0
8.0
9.0
10.0
9.0
9.0
7.0
8.0
8.0
8.0
10.0
8.0
8.0
8.0
9.0
8.0
9.0
8.0
8.0
9.0
9.0
8.0
8.0
8.0
7.0
8.0
7.0
8.0
9.0
8.0
9.0
8.0
9.0
9.0
9.0
8.0
9.0
8.0
8.0
9.0
8.0
8.0
8.0
10.0
9.0
9.0
10.0
a_loss -0.033717394
c_loss 4.3168554
entropy 0.10167711
9.0
8.0
8.0
8.0
8.0
9.0
9.0
8.0
10.0
8.0
8.0
10.0
11.0
8.0
7.0
9.0
10.0
8.0
9.0
7.0
7.0
8.0
9.0
8.0
9.0
9.0
8.0
9.0
8.0
9.0
8.0
11.0
9.0
7.0
8.0
8.0
9.0
9.0
10.0
9.0
8.0
9.0
9.0
10.0
9.0
9.0
9.0
8.0
7.0
8.0
7.0
7.0
9.0
9.0
10.0
8.0
8.0
8.0
8.0
9.0
10.0
8.0
8.0
9.0
7.0
9.0
9.0
8.0
7.0
9.0
10.0
8.0
9.0
9.0
8.0
9.0
11.0
9.0
8.0
9.0
8.0
10.0
8.0
a_loss 0.015040966
c_loss 0.3429188
entropy 0.11669374
8.0
9.0
8.0
8.0
9.0
9.0
8.0
9.0
7.0
8.0
8.0
9.0
8.0
9.0
9.0
9.0
9.0
9.0
8.0
10.0
8.0
9.0
9.0
8.0
9.0
9.0
9.0
8.0
7.0
9.0
9.0
12.0
9.0
8.0
8.0
8.0
8.0
8.0
8.0
8.0
8.0
9.0
8.0
10.0
7.0
9.0
11.0
10

17.0
11.0
46.0
39.0
14.0
20.0
41.0
55.0
13.0
48.0
9.0
13.0
31.0
50.0
14.0
14.0
22.0
19.0
17.0
28.0
9.0
20.0
a_loss -1.5281993
c_loss 39.98842
entropy 0.66201067
32.0
19.0
58.0
13.0
14.0
16.0
13.0
8.0
55.0
15.0
15.0
13.0
8.0
11.0
8.0
9.0
27.0
9.0
20.0
11.0
12.0
12.0
9.0
13.0
13.0
11.0
10.0
8.0
9.0
8.0
8.0
9.0
8.0
8.0
9.0
10.0
8.0
8.0
9.0
7.0
8.0
8.0
12.0
9.0
9.0
10.0
10.0
7.0
10.0
8.0
9.0
8.0
9.0
9.0
9.0
9.0
8.0
11.0
9.0
8.0
11.0
9.0
9.0
12.0
9.0
10.0
11.0
8.0
8.0
8.0
8.0
a_loss -0.18728504
c_loss 15.320234
entropy 0.3832576
9.0
10.0
10.0
11.0
9.0
10.0
7.0
9.0
9.0
11.0
13.0
9.0
8.0
10.0
14.0
9.0
9.0
9.0
13.0
9.0
8.0
10.0
9.0
10.0
10.0
9.0
11.0
10.0
9.0
8.0
10.0
11.0
9.0
8.0
12.0
9.0
10.0
7.0
13.0
15.0
12.0
8.0
8.0
8.0
9.0
9.0
11.0
22.0
14.0
9.0
10.0
10.0
12.0
10.0
8.0
9.0
14.0
8.0
9.0
8.0
11.0
9.0
9.0
8.0
10.0
9.0
13.0
15.0
11.0
10.0
11.0
7.0
a_loss -0.2459533
c_loss 12.854918
entropy 0.45505682
11.0
9.0
11.0
8.0
12.0
9.0
12.0
9.0
12.0
13.0
11.0
11.0
11.0
7.0
9.0
11.0
10.0
11.0
13.0
13.

...

45.0
32.0
19.0
24.0
14.0
14.0
42.0
26.0
14.0
9.0
72.0
20.0
10.0
78.0
34.0
13.0
13.0
72.0
11.0
19.0
38.0
23.0
14.0
a_loss 0.11866954
c_loss 0.3456233
entropy 0.6534898
37.0
15.0
14.0
18.0
11.0
25.0
20.0
14.0
16.0
17.0
32.0
12.0
20.0
29.0
17.0
25.0
18.0
19.0
16.0
15.0
12.0
25.0
18.0
16.0
22.0
20.0
31.0
20.0
20.0
11.0
18.0
14.0
20.0
44.0
16.0
17.0
a_loss 0.47662383
c_loss 0.9100083
entropy 0.6487434
30.0
23.0
15.0
40.0
26.0
25.0
31.0
20.0
11.0
17.0
18.0
18.0
13.0
24.0
23.0
35.0
23.0
18.0
17.0
36.0
15.0
34.0
31.0
14.0
28.0
43.0
21.0
43.0
19.0
a_loss 0.31668824
c_loss 0.3156047
entropy 0.6517267
32.0
18.0
30.0
66.0
43.0
30.0
15.0
71.0
25.0
40.0
25.0
27.0
27.0
101.0
75.0
30.0
a_loss -0.6155599
c_loss 44.465717
entropy 0.67332405
45.0
54.0
70.0
52.0
94.0
79.0
24.0
39.0
25.0
13.0
27.0
51.0
54.0
48.0
15.0
64.0
14.0
18.0
10.0
12.0
11.0
14.0
9.0
18.0
18.0
19.0
10.0
24.0
41.0
38.0
a_loss 0.17140128
c_loss 0.5857606
entropy 0.6445128
32.0
26.0
24.0
25.0
40.0
25.0
38.0
20.0
12.0
24.0
24.0
22.0
58.0


144.0
26.0
47.0
19.0
17.0
66.0
24.0
41.0
24.0
23.0
33.0
28.0
92.0
19.0
a_loss 0.23635702
c_loss 0.23998865
entropy 0.6135271
45.0
62.0
28.0
19.0
81.0
79.0
43.0
60.0
17.0
52.0
8.0
20.0
24.0
37.0
19.0
11.0
14.0
25.0
20.0
39.0
23.0
16.0
14.0
14.0
12.0
10.0
17.0
25.0
17.0
21.0
16.0
17.0
12.0
20.0
a_loss 0.20234758
c_loss 0.70795107
entropy 0.6081801
17.0
13.0
10.0
17.0
12.0
16.0
9.0
14.0
12.0
12.0
14.0
11.0
20.0
14.0
12.0
18.0
21.0
10.0
9.0
13.0
8.0
11.0
18.0
12.0
24.0
14.0
13.0
20.0
10.0
19.0
26.0
12.0
16.0
9.0
11.0
12.0
13.0
18.0
15.0
19.0
15.0
21.0
16.0
22.0
18.0
18.0
15.0
13.0
20.0
a_loss 0.17948738
c_loss 0.8445533
entropy 0.6293565
32.0
13.0
12.0
12.0
17.0
26.0
16.0
12.0
25.0
20.0
21.0
22.0
24.0
22.0
12.0
12.0
21.0
20.0
30.0
15.0
12.0
24.0
19.0
26.0
43.0
45.0
23.0
53.0
40.0
34.0
43.0
a_loss 0.26869056
c_loss 0.48345444
entropy 0.6127502
17.0
18.0
31.0
33.0
19.0
19.0
29.0
9.0
21.0
32.0
22.0
86.0
16.0
27.0
35.0
28.0
11.0
22.0
23.0
17.0
13.0
13.0
33.0
19.0
38.0
14.0
12.0
13.0
a_loss 0.4

115.0
a_loss 0.10248607
c_loss 0.0873737
entropy 0.6205658
100.0
82.0
91.0
88.0
98.0
102.0
58.0
158.0
a_loss 0.17053904
c_loss 0.22439042
entropy 0.5966321
69.0
107.0
139.0
110.0
71.0
85.0
52.0
149.0
16.0
12.0
21.0
44.0
a_loss 0.07283533
c_loss 0.12721585
entropy 0.5841068
45.0
13.0
87.0
20.0
45.0
21.0
68.0
15.0
298.0
16.0
19.0
14.0
10.0
23.0
17.0
20.0
96.0
20.0
9.0
37.0
31.0
32.0
39.0
25.0
46.0
10.0
16.0
22.0
13.0
12.0
16.0
a_loss -0.2227917
c_loss 25.67416
entropy 0.6035294
23.0
21.0
24.0
20.0
21.0
24.0
14.0
14.0
25.0
24.0
8.0
12.0
14.0
12.0
13.0
21.0
33.0
15.0
21.0
22.0
12.0
15.0
88.0
17.0
15.0
20.0
11.0
11.0
19.0
18.0
17.0
12.0
16.0
13.0
12.0
11.0
14.0
15.0
10.0
22.0
15.0
12.0
17.0
12.0
11.0
a_loss 0.6938871
c_loss 1.4628351
entropy 0.53010094
14.0
12.0
11.0
25.0
18.0
18.0
19.0
23.0
21.0
12.0
25.0
15.0
18.0
33.0
20.0
21.0
16.0
26.0
10.0
19.0
39.0
27.0
20.0
18.0
33.0
48.0
13.0
28.0
30.0
20.0
18.0
20.0
18.0
a_loss -0.0780162
c_loss 14.208427
entropy 0.58367765
20.0
14.0
24.0
23.0
15.

106.0
115.0
92.0
25.0
98.0
66.0
26.0
23.0
60.0
42.0
21.0
14.0
a_loss 0.22486274
c_loss 0.22640958
entropy 0.60530305
66.0
94.0
78.0
63.0
44.0
86.0
78.0
105.0
38.0
88.0
19.0
55.0
a_loss 0.26615348
c_loss 0.20973346
entropy 0.62427455
128.0
a_loss 0.042755198
c_loss 0.06733924
entropy 0.6447371
191.0
23.0
155.0
161.0
165.0
36.0
190.0
27.0
198.0
59.0
41.0
55.0
a_loss 0.14154108
c_loss 0.16712904
entropy 0.62022984
100.0
84.0
58.0
87.0
138.0
73.0
39.0
a_loss 0.11982421
c_loss 0.20522377
entropy 0.5932346
124.0
361.0
78.0
83.0
110.0
82.0
34.0
a_loss 0.18977231
c_loss 0.15020041
entropy 0.61777174
87.0
92.0
157.0
88.0
28.0
105.0
112.0
223.0
a_loss 0.12153627
c_loss 0.0987021
entropy 0.6157261
145.0
75.0
108.0
118.0
146.0
a_loss 0.11796863
c_loss 0.05004858
entropy 0.59585685
177.0
143.0
110.0
130.0
36.0
143.0
32.0
72.0
22.0
25.0
157.0
22.0
34.0
31.0
71.0
63.0
26.0
26.0
a_loss -0.6934898
c_loss 50.949097
entropy 0.5342255
52.0
52.0
64.0
34.0
33.0
78.0
78.0
26.0
24.0
64.0
40.0
24.0
40.0
31.0
2

KeyboardInterrupt: 

In [ ]:
observation = env.reset()
cum_reward = 0
frames = []
for t in range(5000):
    # Render into buffer. 
    frames.append(env.render(mode = 'rgb_array'))
    action = agent.agent.get_action(observation)
    observation, reward, done, info = env.step(action)
    cum_reward += reward
    if done:
        print(cum_reward)
        break
env.close()
display_frames_as_gif(frames)